In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

2024-01-19 17:51:42.085751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Cargar datos
df = pd.read_csv("juadores_fantasy.csv.csv")

print(df.columns)



Index(['Nombre', 'Precio', 'Media', 'Total_puntos', 'posicion', 'equipo'], dtype='object')


In [3]:
# Dividir los datos en características (X) y el objetivo (y)
X = df[['Precio', 'Media']]
y_puntos = df['Total_puntos']
y_precio = df['Precio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.2, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [5]:
# Crear el modelo para predecir precio
model_precio = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

model_precio.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir precio
model_precio.fit(X_train_scaled, y_precio_train, epochs=50, batch_size=32, verbose=0)


In [6]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Obtener el jugador con más puntos y el jugador con el precio más alto
indice_max_puntos = puntos_predictions.argmax()
indice_max_precio = precio_predictions.argmax()

jugador_mas_puntos = df.loc[indice_max_puntos, 'Nombre']
jugador_mas_precio = df.loc[indice_max_precio, 'Nombre']

print(f"Jugador con más puntos predichos: {jugador_mas_puntos}")
print(f"Jugador con precio más alto predicho: {jugador_mas_precio}")


4/4 [==============================] - 0s 2ms/step
Jugador con más puntos predichos: R. Alcaraz
Jugador con precio más alto predicho: R. Alcaraz


In [7]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Obtener los 10 jugadores con más puntos predichos
top_10_jugadores_puntos = df.nlargest(10, 'Total_puntos')

# Obtener los 10 jugadores con los precios más altos predichos
top_10_jugadores_precio = df.nlargest(10, 'Precio')

print("Top 10 jugadores con más puntos predichos:")
print(top_10_jugadores_puntos[['Nombre', 'Total_puntos']])

print("\nTop 10 jugadores con precios más altos predichos:")
print(top_10_jugadores_precio[['Nombre', 'Precio']])



4/4 [==============================] - 0s 2ms/step
Top 10 jugadores con más puntos predichos:
          Nombre  Total_puntos
0  J. Bellingham           173
1        T. Kubo           147
2   A. Griezmann           144
3    I. Williams           139
4     B. Mayoral           138
5      A. García           137
6      A. Dovbyk           130
7      Á. Valles           128
8         Pepelu           127
9     A. Budimir           127

Top 10 jugadores con precios más altos predichos:
            Nombre    Precio
2     A. Griezmann  24384000
0    J. Bellingham  24262000
56  R. Lewandowski  20994000
1          T. Kubo  20904000
5        A. García  20771000
18     F. Valverde  20115000
10            Isco  19605000
13         Rodrygo  19338000
3      I. Williams  18358000
22       G. Moreno  18341000


In [8]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_scaled)
precio_predictions = model_precio.predict(X_test_scaled)

# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)
mse_precio = mean_squared_error(y_precio_test, precio_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)
r2_precio = r2_score(y_precio_test, precio_predictions)

print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

print(f"MSE para precio: {mse_precio:.2f}")
print(f"R^2 para precio: {r2_precio:.2f}")


4/4 [==============================] - 0s 8ms/step
MSE para puntos: 388.33
R^2 para puntos: 0.69
MSE para precio: 58078977408543.36
R^2 para precio: -0.59


In [9]:
# Crear un DataFrame con las predicciones
predicciones_df = pd.DataFrame({
    'Nombre': df.iloc[X_test.index]['Nombre'],  # Recuperar los nombres de los jugadores
    'Posición': df['Posición'][y_train],   # Utilizar la posici
    'Predicciones_Puntos': puntos_predictions.flatten(),  # Aplanar el array de predicciones
    'Predicciones_Precio': precio_predictions.flatten()  # Aplanar el array de predicciones
})

In [12]:
predicciones_df

,Nombre,Predicciones_Puntos,Predicciones_Precio
304,M. Morlanes,20.198320,152.637390
497,M. Tejón,-1.837716,144.140076
440,F. Coquelin,16.766140,148.604401
153,Á. Berenguer,44.402699,604.990051
499,A. López,-1.836936,144.151184
...,...,...,...
397,R. Fernández,36.473633,271.145355
167,A. Suárez,46.842648,534.593323
275,L. Hernández,54.690594,581.942383
505,J. Doménech,-1.835376,144.173370


In [10]:
# Guardar las predicciones en un archivo CSV
predicciones_df.to_csv('predicciones.csv', index=False)

In [11]:
joblib.dump(model_puntos, 'redes_neuronales.pkl')

['redes_neuronales.pkl']